In [1]:
%matplotlib widget
import matplotlib.pyplot as plt 
from smpr3d.core import Sparse4DData, Metadata4D
from pathlib import Path
from smpr3d.util import advanced_raster_scan
from ipywidgets import AppLayout, FloatSlider, GridspecLayout, VBox, Tab, Box, HBox, IntSlider
plt.ioff()

In [2]:
# from smpr3d.core import ReconstructionOptions, reconstruct

# metadata2 = Metadata4D(E_ev = E_ev, 
#                        alpha_rad = metadata.alpha_rad, 
#                        dr=metadata.dr,
#                        k_max = k_max,
#                        rotation_deg = t.rotation_deg)

# options = ReconstructionOptions()

# out = sm.reconstruct(data, metadata, options)

# S = out.smatrix
# r = out.r
# Psi = out.Psi
# R_factor = out.R_factors

In [3]:
scan_number = 147
base_path = Path('/home/philipp/nvme/insync_berkeley/2020-09-22/')
results_path = base_path / 'results/'
fp = results_path / 'ptycho' / f'{scan_number:3d}_dense.h5'

meta = Metadata4D.from_h5(fp,'meta')
data= Sparse4DData.from_h5(fp,'data')


In [4]:
meta

Metadata4D(scan_step=array([0.31626087, 0.31626087], dtype=float32), pixel_step=array([1.02852336, 1.02852336]), k_max=array([1.62606797, 1.62606797]), alpha_rad=0.025, rotation_deg=0.0, E_ev=80000.0, wavelength=0.041757171951176904, aberrations=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [5]:
data

Sparse4DData(scan_dimensions=array([512, 512]), frame_dimensions=array([48, 48]))

In [6]:
ddense = Sparse4DData.fftshift(data).to_dense(1)

slic_edge = np.s_[:,-50:,:,:]
sds = np.sum(ddense[slic_edge],(0,1)) / np.prod(data.scan_dimensions)
vacuum_probe = sds * (sds > sds.max() * 20e-2) 

fig,ax = plt.subplots()
ax.imshow(vacuum_probe)
AppLayout(center=fig.canvas)

radius_data_int : 24 
radius_max_int  : 24 
Dense frame size: 48x 48
sparse_to_dense_datacube_crop_gain_mask dtypes: torch.float32 torch.int16 torch.float32 int64


AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [7]:
from smpr3d.util import ZernikeProbeSingle
from smpr3d.util import *

dx = 1/2/meta.k_max

qnp = fourier_coordinates_2D(data.frame_dimensions, dx, centered=False)
q = th.as_tensor(qnp)
Psi_gen = ZernikeProbeSingle(q, meta.wavelength, fft_shifted=True)
Ap0 = th.as_tensor(fftshift(vacuum_probe))
C1 = th.as_tensor(meta.aberrations)
Psi_model = Psi_gen(C1, Ap0).detach()
Psi = Psi_model
psi = th.fft.ifft2(Psi_model,norm='ortho')
fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].imshow(np.angle(psi))
ax[1].imshow(np.abs(psi))

# d['probe_real'] = psi.real
# d['probe_imag'] = psi.imag

AppLayout(center=fig.canvas)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [8]:
q

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0678,  0.0678,  0.0678,  ...,  0.0678,  0.0678,  0.0678],
         [ 0.1355,  0.1355,  0.1355,  ...,  0.1355,  0.1355,  0.1355],
         ...,
         [-0.2033, -0.2033, -0.2033,  ..., -0.2033, -0.2033, -0.2033],
         [-0.1355, -0.1355, -0.1355,  ..., -0.1355, -0.1355, -0.1355],
         [-0.0678, -0.0678, -0.0678,  ..., -0.0678, -0.0678, -0.0678]],

        [[ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678],
         [ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678],
         [ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678],
         ...,
         [ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678],
         [ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678],
         [ 0.0000,  0.0678,  0.1355,  ..., -0.2033, -0.1355, -0.0678]]])

In [9]:
from smpr3d.util import imsave
Ap = Ap0.cuda()
q = q.cuda()

Psi_gen = ZernikeProbeSingle(q, meta.wavelength, fft_shifted=True)

C_target = th.zeros((12)).float().cuda()
C_target[0] = 0

C_model = th.tensor(meta.aberrations).float().cuda()
C_model[0] = 0

psi_target = Psi_gen(C_target, Ap)
psi_model = Psi_gen(C_target, Ap)
psi_model = psi_model.cuda()
psi_model.requires_grad_(False)

f, ax = plt.subplots()
ax.imshow(imsave(th.fft.ifft2(psi_model, norm='ortho').cpu().numpy()))
AppLayout(center=f.canvas)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [10]:
data

Sparse4DData(scan_dimensions=array([512, 512]), frame_dimensions=array([48, 48]))

In [11]:
# %%
size = [60,180]
start = [0,60]
d2 = data.slice(np.s_[start[0]:start[0] + size[0], start[1]:start[1] + size[1]])

f, ax = plt.subplots()
ax.imshow(d2.sum_diffraction())
AppLayout(center=f.canvas)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [12]:
d2, dx


(Sparse4DData(scan_dimensions=array([ 60, 180]), frame_dimensions=array([48, 48])),
 array([0.30749022, 0.30749022]))

In [13]:
import torch.optim as optim
from smpr3d.util import array_split_divpoints_ntotal
r = advanced_raster_scan(d2.scan_dimensions[0], d2.scan_dimensions[1], fast_axis=1, mirror=[1, 1], theta=meta.rotation_deg, dy=meta.pixel_step[0], dx=meta.pixel_step[1])
margin = 10
M = th.tensor(d2.frame_dimensions).int()
N = th.tensor(np.ceil(r.max(axis=0)).astype(np.int32)) + margin + M
K = r.shape[0]

print('N:', N)
print('M:', M)
print('K:', K)

T_model = th.ones(tuple(N), requires_grad=True, device=th.device('cuda:0'), dtype=th.complex64)

pos = th.as_tensor(r + margin/2, device=T_model.device)

indices_target = th.as_tensor(d2.indices, device=T_model.device)
counts_target = th.as_tensor(d2.counts, device=T_model.device)
ish = indices_target.shape

indices_target = indices_target.view((K, ish[-1]))
counts_target = th.sqrt(counts_target.view((K, ish[-1])).type(th.float32))

C_model.requires_grad = False
pos.requires_grad = False
# optimizer = optim.Adam([T_model, C_model, pos], lr=1e-2)
optimizer = optim.Adam([T_model], lr=1e-2)
# loss_function = F.MSELoss(reduction='sum')
# loss_function = F.PoissonNLLLoss(log_input=False, reduction='sum')



N: tensor([119, 243], dtype=torch.int32)
M: tensor([48, 48], dtype=torch.int32)
K: 10800


In [14]:
import torch as th
import numpy as np
from torch.autograd import Function
from numba import cuda
from numpy.fft import fftfreq

@cuda.jit
def sparse_amplitude_loss_kernel(a_model, indices_target, counts_target, loss, grad, frame_dimensions,
                                 no_count_indicator):
    k = cuda.grid(1)
    K, _ = indices_target.shape
    MY, MX = frame_dimensions
    if k < K:
        for i in range(indices_target[k].shape[0]):
            idx1d = indices_target[k, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            if idx1d != no_count_indicator:
                grad[k, my, mx] = 1 - (counts_target[k, i] / a_model[k, my, mx])
                cuda.atomic.add(loss, (0), (a_model[k, my, mx] - counts_target[k, i]) ** 2)


def sparse_amplitude_loss(a_model, indices_target, counts_target, frame_dimensions):
    """

    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (1,), grad (K x M1 x M2)
    """
    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(indices_target.shape[0]) / threadsperblock).astype(np.int))

    loss = th.zeros((1,), device=a_model.device, dtype=th.float32)
    grad = th.ones_like(a_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max
    sparse_amplitude_loss_kernel[blockspergrid, threadsperblock](a_model.detach(), indices_target.detach(),
                                                                 counts_target.detach(), loss.detach(), grad.detach(),
                                                                 frame_dimensions, no_count_indicator)
    return loss, grad

class SparseAmplitudeLoss(Function):

    @staticmethod
    def forward(ctx, a_model, indices_target, counts_target):
        frame_dimensions = th.as_tensor(a_model.shape[1:], device=a_model.device)
        loss, grad = sparse_amplitude_loss(a_model, indices_target, counts_target, frame_dimensions)
        loss.requires_grad = True
        ctx.save_for_backward(grad)
        return loss

    @staticmethod
    def backward(ctx, *grad_outputs):
        grad_input, = ctx.saved_tensors
        grad_indices = None
        grad_counts = None
        return grad_input, grad_indices, grad_counts

class PtychoSubpix(th.autograd.Function):
    @staticmethod
    def forward(ctx, S: th.Tensor, M: th.Tensor, Psi, pos: th.Tensor) -> th.Tensor:
        """
        :param S: N1 x N2 x 2     tensor
        :param psi: M1 x M2 x 2     tensor in fourier space
        :param pos: K x 2 real                  tensor

        :return: D x K x M1 x M2                measured amplitudes in the far field
        """
        qy, qx = np.meshgrid(fftfreq(M[0].item()), fftfreq(M[1].item()), indexing='ij')
        # M1 x M2 x 2
        q = th.stack([th.as_tensor(qy), th.as_tensor(qx)]).float().cuda()
        rs = pos - pos.int()
        ramp = th.exp( -2j * np.pi * (q[0][None, ...] * rs[:, 0][:, None, None] + q[1][None, ...] * rs[:, 1][:, None, None]))
        # K x B x M x M x 2
        swap = th.LongTensor([1, 0])
        pos2 = pos[:, swap]
        # K x M1 x M2 x 2
        frames_exit = gather_patches(S, axes=[0, 1], positions=pos2.long(), patch_size=M, out=None)
        # frames_exit = th.stack([S[:, r[1]:r[1] + M[1].item(), r[0]:r[0] + M[0].item()] for r in pos], 0)
        # K x x M x M x 2
        # psi = th.fft(psi, 2)
        Psi = Psi.unsqueeze(0).repeat(ramp.shape[0], 1, 1, 1)
        psi = th.fft.ifft2(Psi * ramp, norm='ortho')
        # psi = th.ifft(Psi, 2, True)
        # plotcxmosaic(complex_numpy(psi.detach().cpu()))
        psi_exit = frames_exit * psi
        # K x M x M x 2
        Psi = th.fft.fft2(psi_exit, norm='ortho')
        # K x M x M x 2
        a_model = th.abs(Psi)

        ctx.save_for_backward(frames_exit, Psi, psi, pos2, ramp, th.Tensor([S.shape])[0].int())

        return a_model

    def backward(ctx, grad_output):
        frames_exit, Psi_exit, psi, pos2, ramp, ss = ctx.saved_tensors

        M = [grad_output.shape[1], grad_output.shape[2]]

        # K x M1 x M2 x 2
        grad_Psi = Psi_exit * grad_output
        # K x M1 x M2 x 2
        grad_Psi = th.fft.ifft2(grad_Psi, norm='ortho')
        # M1 x M2 x 2

        tmp = grad_Psi * psi.conj()
        grad_S = th.zeros(tuple(ss.numpy()), device=Psi_exit.device, dtype=th.float32)
        grad_S = scatter_add_patches(tmp, grad_S, [0, 1], pos2.long(), M, reduce_dim=None)
        # same = th.sum((grad_S[re] == grad_S[im]) * (grad_S[re] != 0)* (grad_S[im] != 0))
        # print(f'same: {same}')
        # zplot([frames_exit_shifted[10, :, :, 0].cpu(), frames_exit_shifted[10, :, :, 1].cpu()], cmap=['inferno', 'inferno'], figsize=(9, 5))
        # zplot([frames_exit_shifted[20, :, :, 0].cpu(), frames_exit_shifted[20, :, :, 1].cpu()], cmap=['inferno', 'inferno'], figsize=(9, 5))
        grad_psi = grad_Psi * frames_exit.conj()
        grad_psi = th.fft2(grad_psi, 2, True)
        ramp = ramp.view_as_real()
        ramp[...,1] *= -1 
        ramp = ramp.view_as_complex()
        # K x M1 x M2 x 2
        grad_psi = th.fft.ifft2(grad_psi * ramp, norm='ortho')
        grad_psi = th.sum(grad_psi, 0)
        grad_Psi = th.fft.fft2(grad_psi, norm='ortho')
        # print('i got here 2')

        grad_M = None
        grad_pos = None

        return grad_S, grad_M, grad_Psi, grad_pos

In [15]:
A = PtychoSubpix.apply
sparse_amplitude_loss_funtion = SparseAmplitudeLoss.apply

In [16]:
th.Tensor([T_model.shape])[0].int()

tensor([119, 243], dtype=torch.int32)

In [17]:
T_model.dtype

torch.complex64

In [18]:
from tqdm import trange
i = 0
it = 1
it = 30

n_batches = 4
divpoints = array_split_divpoints_ntotal(K, n_batches)

# for i in trange(it):
sum_loss = 0
random_order = th.randperm(K)
#     for b in range(n_batches):
b = 1
take_ind = random_order[divpoints[b]:divpoints[b+1]]
if i > 10:
    C_model.requires_grad = True

# print(pos[100])
# if i == 100:
#     t = T_model.clone().detach().cpu().numpy()
#     zplot([t[re], t[im]], cmap=['inferno', 'inferno'], figsize=(9, 5))
optimizer.zero_grad()
#         if i == 0:
#             Ap1 = Ap + th.randn_like(Ap) *1e-4
#         else:
Ap1 = Ap.float()
Psi_model = Psi_gen(C_model, Ap1)
a_model = A(T_model, M[:2], Psi_model, pos[take_ind])



RuntimeError: CUDA error: device-side assert triggered

In [ ]:
# a_model = A(T_model, a, pos)
loss = sparse_amplitude_loss_funtion(a_model, indices_target[take_ind], counts_target[take_ind])
sum_loss += loss.item()
loss.backward()

In [ ]:
# if i > 10:
    # C_model.grad[1:] = 0
# # print(pos.grad)
# # print(psi_model.grad.sum())
# t = complex_numpy(T_model.grad.clone().cpu())
# t = rotate(t.real, -theta) + 1j * rotate(t.real, -theta)
# plotcx(t[m:-m,m:-m])
# zplot([t.real[m:-m, m:-m], t.imag[m:-m, m:-m]], title=['T_model.grad Re', 'T_model.grad Im'], cmap=['inferno', 'inferno'], figsize=(9, 5))
optimizer.step()
print(f'i {i} loss {sum_loss}, C_model = {C_model[0]} , C_target = {C_target[0]}')
#%%
from skimage.transform import rotate
m = 80
t = complex_numpy(T_model.clone().detach().cpu())
t = rotate(t.real,-theta) + 1j * rotate(t.imag,-theta)
# plotcx(t[m:-m,m:-m])
zplot([np.abs(t)[m:-m,m:-m], np.angle(t)[m:-m,m:-m]], title=['Abs', 'Angle'], cmap=['inferno', 'inferno'], figsize=(9, 5))

In [15]:
it

NameError: name 'it' is not defined